<a href="https://colab.research.google.com/github/crdsteixeira/OR_project/blob/main/Scheduling_Aircraft_Landings_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download the library

In [1]:
!pip install docplex
!pip install cplex

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 640.2/640.2 KB 16.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docplex: filename=docplex-2.24.232-py3-none-any.whl size=682306 sha256=6851c04939ad647c0831330bde555bb78180291b7e9050522e90c2257401e23c
  Stored in directory: /root/.cache/pip/wheels/cd/84/5d/b9c307d9cf361c49d41ddea36761e226bba3afdfd038673dcd
Successfully built docplex
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 MB 17.3 MB/s eta 0:00:00


In [2]:
from docplex.cp.model import *
import numpy as np

# Info

The format of these data files is:
number of planes (p), freeze time
for each plane i (i=1,...,p):
   - appearance time, 
   - earliest landing time, 
   - target landing time,
   - latest landing time, 
   - penalty cost per unit of time for landing before target, 
   - penalty cost per unit of time for landing after target

for each plane j (j=1,...p): separation time required after 
                                i lands before j can land


### Sigle runway definitions and variables

![image.png](https://github.com/crdsteixeira/OR_project/blob/main/Images/single_run.png?raw=1)


### Multirunway defintions and variables 

![image.png](https://github.com/crdsteixeira/OR_project/blob/main/Images/multi_run.png?raw=1)

## Get data

In [3]:
def read_datafiles(file):
    with open(file, 'r') as data:
        data_lines = data.readlines()
        number_planes = int(data_lines[0].split()[0])
        freeze_time =  int(data_lines[0].split()[1])
        mixed_data = [line.split() for line in data_lines[1:] if not line.isspace()]
        mixed_data = [[(float(j)) for j in i] for i in mixed_data]
        
        flight_details = np.empty([0,6],dtype=float)
        separation_time = np.empty([0,number_planes],dtype=float)
        
        flag = 0
       
        for element in mixed_data:
            if flag == 0: # flight details
                flight_details = np.vstack([flight_details, np.array(element)])
                flag = 1
                element_final = []
            else:  # separation_times
                element_final.extend(element)
                if len(element_final) == number_planes:
                    separation_time = np.vstack([separation_time, np.array(element_final)])
                    flag = 0
                
        print(f" number planes: {number_planes}")
        print(f" freeze time: {freeze_time}")
        print(f" mixed data: {mixed_data}")
        print(f" flight details: {flight_details}")
        print(f" separation time: {separation_time}")
    
    return number_planes, freeze_time, flight_details, separation_time


In [4]:
read_datafiles("./airland1.txt")

 number planes: 10
 freeze time: 10
 mixed data: [[54.0, 129.0, 155.0, 559.0, 10.0, 10.0], [99999.0, 3.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0], [15.0, 15.0], [120.0, 195.0, 258.0, 744.0, 10.0, 10.0], [3.0, 99999.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0], [15.0, 15.0], [14.0, 89.0, 98.0, 510.0, 30.0, 30.0], [15.0, 15.0, 99999.0, 8.0, 8.0, 8.0, 8.0, 8.0], [8.0, 8.0], [21.0, 96.0, 106.0, 521.0, 30.0, 30.0], [15.0, 15.0, 8.0, 99999.0, 8.0, 8.0, 8.0, 8.0], [8.0, 8.0], [35.0, 110.0, 123.0, 555.0, 30.0, 30.0], [15.0, 15.0, 8.0, 8.0, 99999.0, 8.0, 8.0, 8.0], [8.0, 8.0], [45.0, 120.0, 135.0, 576.0, 30.0, 30.0], [15.0, 15.0, 8.0, 8.0, 8.0, 99999.0, 8.0, 8.0], [8.0, 8.0], [49.0, 124.0, 138.0, 577.0, 30.0, 30.0], [15.0, 15.0, 8.0, 8.0, 8.0, 8.0, 99999.0, 8.0], [8.0, 8.0], [51.0, 126.0, 140.0, 573.0, 30.0, 30.0], [15.0, 15.0, 8.0, 8.0, 8.0, 8.0, 8.0, 99999.0], [8.0, 8.0], [60.0, 135.0, 150.0, 591.0, 30.0, 30.0], [15.0, 15.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0], [99999.0, 8.0], [85.0, 160.0, 180.0, 657.0, 30.

(10, 10, array([[ 54., 129., 155., 559.,  10.,  10.],
        [120., 195., 258., 744.,  10.,  10.],
        [ 14.,  89.,  98., 510.,  30.,  30.],
        [ 21.,  96., 106., 521.,  30.,  30.],
        [ 35., 110., 123., 555.,  30.,  30.],
        [ 45., 120., 135., 576.,  30.,  30.],
        [ 49., 124., 138., 577.,  30.,  30.],
        [ 51., 126., 140., 573.,  30.,  30.],
        [ 60., 135., 150., 591.,  30.,  30.],
        [ 85., 160., 180., 657.,  30.,  30.]]), array([[9.9999e+04, 3.0000e+00, 1.5000e+01, 1.5000e+01, 1.5000e+01,
         1.5000e+01, 1.5000e+01, 1.5000e+01, 1.5000e+01, 1.5000e+01],
        [3.0000e+00, 9.9999e+04, 1.5000e+01, 1.5000e+01, 1.5000e+01,
         1.5000e+01, 1.5000e+01, 1.5000e+01, 1.5000e+01, 1.5000e+01],
        [1.5000e+01, 1.5000e+01, 9.9999e+04, 8.0000e+00, 8.0000e+00,
         8.0000e+00, 8.0000e+00, 8.0000e+00, 8.0000e+00, 8.0000e+00],
        [1.5000e+01, 1.5000e+01, 8.0000e+00, 9.9999e+04, 8.0000e+00,
         8.0000e+00, 8.0000e+00, 8.0000e+00, 

# The MIP model

In [18]:
def MIP_model(file_name,R):
    from docplex.mp.model import Model
    mdl = Model("Scheduling Aircraft Landing - Multi Runaway Static Case")
    
    P, freeze_time, flight_details, separation_time = read_datafiles(file_name)
    
    # The first column relates to the actual appearence time of the plane so will not be taken into account for our decision variables

    E = flight_details[:,1]  #earliest landing time,
    T = flight_details[:,2]  #target landing time,
    L = flight_details[:,3]  #latest landing time,
    g = flight_details[:,4]  #penalty cost per unit of time for landing before target,
    h = flight_details[:,5]  #penalty cost per unit of time for landing after target
    range_LT = max(flight_details[:,3]) - min(flight_details[:,1]) #landing range time

    s = separation_time      # here we consider separation time between planes that land in different runways, fundamentally their value is the same as the separation time matrix but in constraints it should be multiplied by the binary variable that says if the planes land in the same runway or not

    ij = [(i,j) for i in np.arange(P) for j in np.arange(P)] # indexes for plane i and j landing in the same runway
    ir = [(i, r) for i in np.arange(P) for r in np.arange(R)] # indexes for plane i landing in runway r

    # Creating the time spaces
    W = []
    V = []
    U = []
    for i in range(P):
      for j in range(P):
        if i != j:
          min_range_i, min_range_j, max_range_i, max_range_j = E[i], E[j], L[i], L[j]
          land_range_i = [min_range_i, max_range_i]
          land_range_j = [min_range_j, max_range_j]

          overlap = max(0, min(land_range_i[1], land_range_j[1]) - max(land_range_i[0], land_range_j[0])) #Check if landing intervals overlap

          # To confirm if the separation time window is safe we should see if latest landing time of plane i plus the separation time  
          # between plane i and j is lower and then the earliest landing time for plane j

          # This way we are also confirming that i lands before j

          if (overlap == 0) and ((L[i] + separation_time[i,j]) < E[j]): # confirm that the windows don't intersept and separation time is big enough
              W.append((i,j))
          else:
            if (L[i] < E[j]): # windows intersept but we know that i lands before j
              V.append((i,j))
            else: 
              U.append((i,j)) # we don't know the order in which one lands
        else: 
          continue
    

    # Defining decision variables

    alpha   = mdl.continuous_var_dict(np.arange(P),lb=0,ub=mdl.infinity, name="alpha") # how soon a plane lands before target time
    beta    = mdl.continuous_var_dict(np.arange(P),lb=0,ub=mdl.infinity, name="beta") # how soon a plane lands after target time
    x       = mdl.continuous_var_dict(np.arange(P),lb=0,ub=mdl.infinity, name="x") # landing time for a plane
    sigma   = mdl.binary_var_dict(ij,lb=0,ub=1, name="sigma") # binary variable: 1 if plane i lands before plane j, 0 elsewhere
    y       = mdl.binary_var_dict(ir,lb=0,ub=1, name="y") # 1 if plane i lands on runway  r , 0 otherwise
    z       = mdl.binary_var_dict(ij,lb=0,ub=1, name="z") # 1 if planes i and  j land on the same runway, 0 otherwise



    # Adding constraints
    
    mdl.add_constraints(x[i]>=E[i] for i in np.arange(P)) # Const 1 - Landing time of plane i must be later than the earliest landing time
    mdl.add_constraints(x[i]<=L[i] for i in np.arange(P)) # Const 1 - Landing time of plane i must be earlier than the before latest landing time
    mdl.add_constraints(sigma[i,j]+sigma[j,i]==1 for i in np.arange(P) for j in np.arange(P) if j!=i)  # Const 2 - Plain i must land before plain j or plain j before plain i
    mdl.add_constraints(alpha[i]>=T[i]-x[i] for i in np.arange(P)) # Const 14 - How soon plane i lands before T[i] must be larger than T[i] - x[i]
    mdl.add_constraints(alpha[i]<=T[i]-E[i] for i in np.arange(P)) # Const 15 - How soon plane i lands before T[i] must be smaller than T[i] - E[i]
    mdl.add_constraints(alpha[i]>= 0 for i in np.arange(P)) # Const 15 - How soon plane i lands before T[i] must be at least zero
    mdl.add_constraints(beta[i]>=x[i]-T[i] for i in np.arange(P))  # Const 16 - How soon plane i lands after T[i] must be larger than x[i] - T[i]
    mdl.add_constraints(beta[i]<=L[i]-T[i] for i in np.arange(P))  # Const 17 - How soon plane i lands after T[i] must be smaller than L[i] - T[i]
    mdl.add_constraints(beta[i]>= 0 for i in np.arange(P))  # Const 17 - How soon plane i lands after T[i] must be at least zero
    mdl.add_constraints(x[i]==T[i]-alpha[i]+beta[i] for i in np.arange(P))  # Const 18 - Landing time is equal to target time minus arriving early or plus arriving late
   
    mdl.add_constraints(mdl.sum(y[i,r] for r in np.arange(R))==1 for i in np.arange(P))     # Const 28 - Plane i can only land in 1 runaway
    mdl.add_constraints(z[i,j]==z[j,i] for i in np.arange(P) for j in np.arange(P) if j>i)  # Const 29 - Symetry constraint: If plane i lands in the same runaway as plane j, plane j lands in the same runaway as plane i
    mdl.add_constraints(z[i,j]>=y[i,r]+y[j,r]-1 for r in np.arange(R) for j in np.arange(P) for i in np.arange(P) if j>i) # Const 30 - If there is any runaway r for which y[i,r]=y[j,r]=1 then z[i,j]=1. If z[i,j]=0 then the planes i and j cannot land on the same runaway 
    
    # Const 6 - If plane i lands before j we know that they belong in subspace W or V 
    for i in range(P):
      for j in range(P):
        if ((i,j) in V) or ((i,j) in W):
          mdl.add_constraints(sigma[i,j] == 1 for i in np.arange(P) for j in np.arange(P))
          
    # Const 12 - Separation time between plane i and plane j must be respected
    for i in range(P):
      for j in range(P):
        if (i,j) in U:
          print(separation_time[i,j] * sigma[j,i])
          mdl.add_constraints(x[j] >= (x[i] + (separation_time[i,j]*sigma[j,i]) - ((L[i] - E[j])*sigma[i,j])) for i in np.arange(P) for j in np.arange(P))  

    # Const 31 - Landing times should respect separation times considering if they land in the same runway or not in subspace V
    for i in range(P):
      for j in range(P):
        if (i,j) in V:
          mdl.add_constraints(x[j] >= (x[i] + separation_time[i,j]*z[i,j] + s[i,j]*(1-z[i,j])) for i in np.arange(P) for j in np.arange(P))

    # Const 33 - Landing times should respect separation times considering if they land in the same runway or not in subspace U
    for i in range(P):
      for j in range(P):
        if (i,j) in U:
          mdl.add_constraints(x[j] >= (x[i] + separation_time[i,j]*z[i,j] + s[i,j]*(1-z[i,j] - (L[i] + max(separation_time[i,j], s[i,j] - E[j])*sigma[i,j]))) for i in np.arange(P) for j in np.arange(P))


    cost_function = mdl.sum(beta[i] * h[i] + alpha[i] * g[i] for i in np.arange(P))

    mdl.minimize(cost_function)
    
    mdl.print_information()

    msol = mdl.solve()
    assert msol is not None, "model can't solve"

    return mdl

In [19]:
file_name = "./airland1.txt"
R = 2

mdl_ = MIP_model(file_name, R)
mdl_.report()
mdl_.print_solution()

 number planes: 10
 freeze time: 10
 mixed data: [[54.0, 129.0, 155.0, 559.0, 10.0, 10.0], [99999.0, 3.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0], [15.0, 15.0], [120.0, 195.0, 258.0, 744.0, 10.0, 10.0], [3.0, 99999.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0], [15.0, 15.0], [14.0, 89.0, 98.0, 510.0, 30.0, 30.0], [15.0, 15.0, 99999.0, 8.0, 8.0, 8.0, 8.0, 8.0], [8.0, 8.0], [21.0, 96.0, 106.0, 521.0, 30.0, 30.0], [15.0, 15.0, 8.0, 99999.0, 8.0, 8.0, 8.0, 8.0], [8.0, 8.0], [35.0, 110.0, 123.0, 555.0, 30.0, 30.0], [15.0, 15.0, 8.0, 8.0, 99999.0, 8.0, 8.0, 8.0], [8.0, 8.0], [45.0, 120.0, 135.0, 576.0, 30.0, 30.0], [15.0, 15.0, 8.0, 8.0, 8.0, 99999.0, 8.0, 8.0], [8.0, 8.0], [49.0, 124.0, 138.0, 577.0, 30.0, 30.0], [15.0, 15.0, 8.0, 8.0, 8.0, 8.0, 99999.0, 8.0], [8.0, 8.0], [51.0, 126.0, 140.0, 573.0, 30.0, 30.0], [15.0, 15.0, 8.0, 8.0, 8.0, 8.0, 8.0, 99999.0], [8.0, 8.0], [60.0, 135.0, 150.0, 591.0, 30.0, 30.0], [15.0, 15.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0], [99999.0, 8.0], [85.0, 160.0, 180.0, 657.0, 30.

DOcplexLimitsExceeded: ignored

## The CP model

In [20]:
def read_datafiles_cp(file):
    with open(file, 'r') as data:
        data_lines = data.readlines()
        number_planes = int(data_lines[0].split()[0])
        freeze_time =  int(data_lines[0].split()[1])
        mixed_data = [line.split() for line in data_lines[1:] if not line.isspace()]
        mixed_data = [[(float(j)) for j in i] for i in mixed_data]
        
        flight_details = np.empty([0,6],dtype=float)
        separation_time = np.empty([0,number_planes],dtype=float)
        
        flag = 0
       
        for element in mixed_data:
            if flag == 0: # flight details
                flight_details = np.vstack([flight_details, np.array(element)])
                flag = 1
                element_final = []
            else:  # separation_times
                element_final.extend(element)
                if len(element_final) == number_planes:
                    separation_time = np.vstack([separation_time, np.array(element_final)])
                    flag = 0

        minimum = min(flight_details[:,1])
        maximum = max(flight_details[:,3])
        num_periods = int(minimum % 10)
        int_size = (maximum-minimum)//num_periods #CT: Adicionei max-min
        period_constraints = []

        x = (0,0)
        lst = list(x)
        for i in range(num_periods-1): #CT: Adicionei -1 
            lst[0], lst[1] = int_size * i, int_size * (i+1)
            x = tuple(lst)
            period_constraints.append(x)

                
        print(f" number planes: {number_planes}")
        print(f" num_periods: {num_periods}")
        print(f" flight details: {flight_details}")
        print(f" period_constraints: {period_constraints}")
    
    return number_planes, num_periods, flight_details, separation_time, period_constraints #CT:added separation time

In [ ]:
read_datafiles_cp("./Data/airland1.txt")

 number planes: 10
 num_periods: 9
 flight details: [[ 54. 129. 155. 559.  10.  10.]
 [120. 195. 258. 744.  10.  10.]
 [ 14.  89.  98. 510.  30.  30.]
 [ 21.  96. 106. 521.  30.  30.]
 [ 35. 110. 123. 555.  30.  30.]
 [ 45. 120. 135. 576.  30.  30.]
 [ 49. 124. 138. 577.  30.  30.]
 [ 51. 126. 140. 573.  30.  30.]
 [ 60. 135. 150. 591.  30.  30.]
 [ 85. 160. 180. 657.  30.  30.]]
 period_constraints: [(0.0, 72.0), (72.0, 144.0), (144.0, 216.0), (216.0, 288.0), (288.0, 360.0), (360.0, 432.0), (432.0, 504.0), (504.0, 576.0)]


(10,
 9,
 array([[ 54., 129., 155., 559.,  10.,  10.],
        [120., 195., 258., 744.,  10.,  10.],
        [ 14.,  89.,  98., 510.,  30.,  30.],
        [ 21.,  96., 106., 521.,  30.,  30.],
        [ 35., 110., 123., 555.,  30.,  30.],
        [ 45., 120., 135., 576.,  30.,  30.],
        [ 49., 124., 138., 577.,  30.,  30.],
        [ 51., 126., 140., 573.,  30.,  30.],
        [ 60., 135., 150., 591.,  30.,  30.],
        [ 85., 160., 180., 657.,  30.,  30.]]),
 array([[9.9999e+04, 3.0000e+00, 1.5000e+01, 1.5000e+01, 1.5000e+01,
         1.5000e+01, 1.5000e+01, 1.5000e+01, 1.5000e+01, 1.5000e+01],
        [3.0000e+00, 9.9999e+04, 1.5000e+01, 1.5000e+01, 1.5000e+01,
         1.5000e+01, 1.5000e+01, 1.5000e+01, 1.5000e+01, 1.5000e+01],
        [1.5000e+01, 1.5000e+01, 9.9999e+04, 8.0000e+00, 8.0000e+00,
         8.0000e+00, 8.0000e+00, 8.0000e+00, 8.0000e+00, 8.0000e+00],
        [1.5000e+01, 1.5000e+01, 8.0000e+00, 9.9999e+04, 8.0000e+00,
         8.0000e+00, 8.0000e+00, 8.0000e+00

In [198]:
from docplex.cp.model import CpoModel

def CP_model(file_name, R):
    # Create a new model
    mdl = CpoModel()

    P, freeze_time, flight_details, separation_time = read_datafiles(file_name)

    # The first column relates to the actual appearence time of the plane so will not be taken into account for our decision variables
    E = flight_details[:,1]  # earliest landing time
    T = flight_details[:,2]  # target landing time
    L = flight_details[:,3]  # latest landing time
    g = flight_details[:,4]  # penalty cost per unit of time for landing before target
    h = flight_details[:,5]  # penalty cost per unit of time for landing after target
    range_LT = max(flight_details[:,3]) - min(flight_details[:,1])  # landing range time

    s = separation_time  # here we consider separation time between planes that land in different runways, fundamentally their value is the same as the separation time matrix but in constraints it should be multiplied by the binary variable that says if the planes land in the same runway or not

    ij = [(i,j) for i in np.arange(P) for j in np.arange(P)]  # indexes for plane i and j landing in the same runway
    ir = [(i, r) for i in np.arange(P) for r in np.arange(R)]  # indexes for plane i landing in runway r

    # Creating the time spaces
    W = []
    V = []
    U = []
    for i in np.arange(P):
      for j in np.arange(P):
        if i != j:
          min_range_i, min_range_j, max_range_i, max_range_j = E[i], E[j], L[i], L[j]
          land_range_i = [min_range_i, max_range_i]
          land_range_j = [min_range_j, max_range_j]

          overlap = max(0, min(land_range_i[1], land_range_j[1]) - max(land_range_i[0], land_range_j[0]))  # Check if landing intervals overlap

          # To confirm if the separation time window is safe we should see if latest landing time of plane i plus the separation time
          # between plane i and j is lower and then the earliest landing time for plane j

          # This way we are also confirming that i lands before j

          if (overlap == 0) and ((L[i] + separation_time[i,j]) < E[j]):  # confirm that the windows don't intersept and separation time is big enough
            W.append((i,j))
          else:
            if (L[i] < E[j]):  # windows intersept but we know that i lands before j
              V.append((i,j))
            else:
              U.append((i,j))  # we don't know the order in which one lands
        else:
          continue

    # Defining decision variables

    alpha = mdl.integer_var_list(P, 0, int(max(L)), name="alpha")  # how soon a plane lands before target time
    beta  = mdl.integer_var_list(P, 0, int(max(L)), name="beta")  # how soon a plane lands after target time
    x = [mdl.integer_var(int(E[i]), int(L[i]), name="x_{}".format(i)) for i in np.arange(P)]  # landing time for a plane

    sigma = mdl.binary_var_list(P, name="sigma")  # binary variable: 1 if plane i lands before plane j, 0 elsewhere
    y = {}
    for i, r in ir:
      y[i, r] = mdl.binary_var(name="y_{}_{}".format(i, r))# 1 if plane i lands on runway  r , 0 otherwise
    
    z = {}
    for i, j in ij:
      z[i, j] = mdl.binary_var(name="z_{}_{}".format(i, j))# 1 if plane i lands on runway  r , 0 otherwise

    # Defining constraints
    for i in np.arange(P):
      mdl.add(x[i] >= E[i]) # Const 1 - Landing time of plane i must be later than the earliest landing time
      mdl.add(x[i] <= L[i]) # Const 1 - Landing time of plane i must be earlier than the before latest landing time
      mdl.add(alpha[i] >= T[i] - x[i]) # Const 14 - How soon plane i lands before T[i] must be larger than T[i] - x[i]
      mdl.add(alpha[i] <= T[i] - E[i]) # Const 15 - How soon plane i lands before T[i] must be smaller than T[i] - E[i]
      mdl.add(alpha[i] >= 0) # Const 15 - How soon plane i lands before T[i] must be at least zero
      mdl.add(beta[i] >= x[i] - T[i])  # Const 16 - How soon plane i lands after T[i] must be larger than x[i] - T[i]
      mdl.add(beta[i] <= L[i] - T[i])  # Const 17 - How soon plane i lands after T[i] must be smaller than L[i] - T[i]
      mdl.add(beta[i] >= 0)  # Const 17 - How soon plane i lands after T[i] must be at least zero
      #mdl.add(x[i] == T[i] - alpha[i] + beta[i])  # Const 18 - Landing time is equal to target time minus arriving early or plus arriving late

    # Const 2 - Plain i must land before plain j or plain j before plain i
    for i in np.arange(P):
      for j in np.arange(P):
        if i != j:
          mdl.add(x[i] == E[i] + alpha[i] - beta[i])
    
    # Const 6 - If plane i lands before j we know that they belong in subspace W or V 
    for i in range(P):
      for j in range(P):
        if ((i,j) in V) or ((i,j) in W):
          mdl.add(sigma[i,j] == 1)
          
    # Const 12 - Separation time between plane i and plane j must be respected
    for i in range(P):
      for j in range(P):
        if (i and j) in U:
          mdl.add(x[j] >= (x[i] + (separation_time[i,j]*sigma[j,i]) - ((L[i] - E[j])*sigma[i,j])))  

    # Const 31 - Landing times should respect separation times considering if they land in the same runway or not in subspace V
    for i in range(P):
      for j in range(P):
        if (i,j) in V:
          mdl.add(x[j] >= (x[i] + separation_time[i,j]*z[i,j] + s[i,j]*(1-z[i,j])))

    # Const 33 - Landing times should respect separation times considering if they land in the same runway or not in subspace U
    for i in range(P):
      for j in range(P):
        if (i and j) in U:
          mdl.add(x[j] >= (x[i] + separation_time[i,j]*z[i,j] + s[i,j]*(1-z[i,j] - (L[i] + max(separation_time[i,j], s[i,j] - E[j])*sigma[i,j]))))

    # Const 28 - Plane i can only land in 1 runaway
    for i in np.arange(P):
        mdl.add(mdl.sum([y[i, r] for r in np.arange(R)]) == 1) 

    # Const 29 - Symetry constraint: If plane i lands in the same runaway as plane j, plane j lands in the same runaway as plane i
    for i in np.arange(P):
      for j in np.arange(P):
        if j > i:
          mdl.add(z[i,j]==z[j,i]) 

    # Const 30 - If there is any runaway r for which y[i,r]=y[j,r]=1 then z[i,j]=1. If z[i,j]=0 then the planes i and j cannot land on the same runaway 
    for i in np.arange(P):
      for j in np.arange(P):
        for r in np.arange(R):
          if j > i:
            mdl.add(z[i,j] >= y[i,r] + y[j,r]-1)


    # Objective function
    mdl.minimize(mdl.sum(g[i] * alpha[i] + h[i] * beta[i] for i in np.arange(P)))

    # Solve model
    solution = mdl.solve()

    # Print solution
    for i in range(P):
      print("Plane {} lands at {}".format(i, solution.get_var_solution(x[i])))


In [199]:
file_name = "./airland1.txt"
R = 2
CP_model(file_name, R)

 number planes: 10
 freeze time: 10
 mixed data: [[54.0, 129.0, 155.0, 559.0, 10.0, 10.0], [99999.0, 3.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0], [15.0, 15.0], [120.0, 195.0, 258.0, 744.0, 10.0, 10.0], [3.0, 99999.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0], [15.0, 15.0], [14.0, 89.0, 98.0, 510.0, 30.0, 30.0], [15.0, 15.0, 99999.0, 8.0, 8.0, 8.0, 8.0, 8.0], [8.0, 8.0], [21.0, 96.0, 106.0, 521.0, 30.0, 30.0], [15.0, 15.0, 8.0, 99999.0, 8.0, 8.0, 8.0, 8.0], [8.0, 8.0], [35.0, 110.0, 123.0, 555.0, 30.0, 30.0], [15.0, 15.0, 8.0, 8.0, 99999.0, 8.0, 8.0, 8.0], [8.0, 8.0], [45.0, 120.0, 135.0, 576.0, 30.0, 30.0], [15.0, 15.0, 8.0, 8.0, 8.0, 99999.0, 8.0, 8.0], [8.0, 8.0], [49.0, 124.0, 138.0, 577.0, 30.0, 30.0], [15.0, 15.0, 8.0, 8.0, 8.0, 8.0, 99999.0, 8.0], [8.0, 8.0], [51.0, 126.0, 140.0, 573.0, 30.0, 30.0], [15.0, 15.0, 8.0, 8.0, 8.0, 8.0, 8.0, 99999.0], [8.0, 8.0], [60.0, 135.0, 150.0, 591.0, 30.0, 30.0], [15.0, 15.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0], [99999.0, 8.0], [85.0, 160.0, 180.0, 657.0, 30.